# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [1]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

In [2]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

df_tolerancia_alta = pd.read_csv("..\\instances\\tolerancia-alta\\indice.csv");
df_tolerancia_baja = pd.read_csv("..\\instances\\tolerancia-baja\\indice.csv");
df_BT_mejor_fact = pd.read_csv("..\\instances\\BT-mejor-fact\\indice.csv");
df_BT_mejor_opt = pd.read_csv("..\\instances\\BT-mejor-opt\\indice.csv");
df_BT_peor_fact = pd.read_csv("..\\instances\\BT-peor-fact\\indice.csv");
df_BT_peor_opt = pd.read_csv("..\\instances\\BT-peor-opt\\indice.csv");
df_dinamica = pd.read_csv("..\\instances\\DP\\indice.csv");


In [3]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)

    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../../negocios", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code is not 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.readline());
    numero_de_podas = int(process.stderr.readline());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion, numero_de_podas;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [4]:
experimentos = [];

# Experimento 1 : Complejidad-BF
Corremos fuerza bruta con las primeras 1000 instancias de tolerancia-baja y tolerancia-alta de contagio.

In [5]:
for n in range(0, 1000):
    fila = df_tolerancia_alta.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "FB", fila["archivo"]]);

for n in range(0, 1000):
    fila = df_tolerancia_baja.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "FB", fila["archivo"]]);

# Experimento 2 : Complejidad-BT
Corremos backtracking con las instancias de mejor y peor caso para poda por factibilidad y por optimalidad. Para mejor caso en ambas, corremos las primeras 1000, y para las de peor caso corremos las primeras 40.

In [6]:
# BT mejor caso con poda por factibilidad
for n in range(0, 10000):
    fila = df_BT_mejor_fact.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-F", fila["archivo"]]);
# BT mejor caso con poda por optimalidad 1
for n in range(0, 10000):
    fila = df_BT_mejor_opt.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O1", fila["archivo"]]);
# BT peor caso con poda por factibilidad
for n in range(0, 40):
    fila = df_BT_peor_fact.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-F", fila["archivo"]]);
# BT peor caso con poda por optimalidad 1
for n in range(0, 40):
    fila = df_BT_peor_opt.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O1", fila["archivo"]]);

# Experimento 3: Complejidad-DP
Corremos todas las instancias para dinamica

In [7]:
for n in range(df_dinamica.shape[0]):
    fila = df_dinamica.iloc[n]
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "DP", fila["archivo"]]);

# Experimento 4: Efectividad de las podas
Corremos bactracking con cada una de las posibles podas activadas para las primeras 40 instancias de tolerancia-baja y tolerancia-alta. 
Vamos a estar comparando el tiempo y la cantidad de nodos recorridos.

In [5]:
# tolerancia baja con poda por factibilidad
for n in range (0, 1200):
    fila = df_tolerancia_baja.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-F", fila["archivo"]]);
# tolerancia baja con poda por optimalidad 1
for n in range (0, 1200):
    fila = df_tolerancia_baja.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O1", fila["archivo"]]);
# tolerancia baja con poda por optimalidad 2 (optimalidad 1 refinada)
for n in range (0, 1200):
    fila = df_tolerancia_baja.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O2", fila["archivo"]]);
# tolerancia baja con poda por optimalidad 3 (optimalidad 2 refinada)
for n in range(0, 1200):
    fila = df_tolerancia_baja.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O3", fila["archivo"]]);
# tolerancia alta con poda por factibilidad
for n in range (0, 1200):
    fila = df_tolerancia_alta.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-F", fila["archivo"]]);
# tolerancia alta con poda por optimalidad 1
for n in range (0, 1200):
    fila = df_tolerancia_alta.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O1", fila["archivo"]]);
# tolerancia alta con poda por optimalidad 2 (optimalidad 1 refinada)
for n in range (0, 1200):
    fila = df_tolerancia_alta.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O2", fila["archivo"]]);
# tolerancia alta con poda por optimalidad 3 (optimalidad 2 refinada)
for n in range(0, 1200):
    fila = df_tolerancia_alta.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT-O3", fila["archivo"]]);

# Experimento 5: Comparacion de DP vs BT con todas las podas
Corremos DP y BT con todas las podas activadas para las primeras 40 instancias de tolerancia-alta y tolerancia-baja. Tambien corremos BT para las primeras 4000 instancias de dinamica.

In [5]:
# backtracking para tolerancia baja
for n in range(0, 3000):
    fila = df_tolerancia_baja.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT", fila["archivo"]]);
# backtracking para tolerancia alta
for n in range(0, 6000):
    fila = df_tolerancia_alta.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT", fila["archivo"]]);
# dinamica para tolerancia baja
for n in range(0, 3000):
    fila = df_tolerancia_baja.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "DP", fila["archivo"]]);
# dinamica para tolerancia alta
for n in range(0, 6000):
    fila = df_tolerancia_alta.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "DP", fila["archivo"]]);
# backtracking para las primeras 500 instancias del set de dinamica
for n in range (0, 1000):
    fila = df_dinamica.iloc[n];
    experimentos.append([fila["dataset"], fila["negocios"], fila["C"], fila["i"], "BT", fila["archivo"]]);

In [6]:
columnas = ["dataset", "negocios", "C", "i", "metodo", "tiempo","nodos_promedio"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    nodos_totales = []
    for i in range(0, T):
        tiempo_de_ejecucion, nodos_recorridos = correr_experimento(experimento[4], experimento[5]);
        tiempos.append(tiempo_de_ejecucion);
        nodos_totales.append(nodos_recorridos);
    nodos_promedio = np.median(nodos_totales);
    tiempo = np.median(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], experimento[4], tiempo, nodos_promedio]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("..\\results\\resultado.csv", index=False, header=True);

'Experimento: 2400/2400'

In [11]:

df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("..\\results\\resultado.csv", index=False, header=True);